In [1]:
# import libraries
import requests
import pandas as pd
import time
from google.colab import  drive

In [2]:
def fetch_video_info(vid_ID):
    # statistics requests
    url_video_stats = 'https://www.googleapis.com/youtube/v3/videos?id='+vid_ID+'&part=statistics&key='+API_KEY
    res_video_stats = requests.get(url_video_stats).json()
    time.sleep(1)

    # parse stats requests to get counts
    view_count = res_video_stats['items'][0]['statistics']['viewCount']
    try:
      like_count = res_video_stats['items'][0]['statistics']['likeCount']
    except KeyError:
      like_count = res_video_stats.get('likeCount', 0)
    try:
      comment_count = res_video_stats['items'][0]['statistics']['commentCount']
    except KeyError:
      comment_count = res_video_stats.get('commentCount', 0)
    
    return view_count, like_count, comment_count

In [3]:
def get_videos(res, df):
  for video in res['items']:
    if video['snippet']['resourceId']['kind'] == 'youtube#video':
      vid_ID = video['contentDetails']['videoId']
      channel_title = video['snippet']['videoOwnerChannelTitle']
      description = video['snippet']['description']
      time_published = str(video['snippet']['publishedAt']).split('T')[0]
      title = video['snippet']['title']

      view_count, like_count, comment_count = fetch_video_info(vid_ID)

      df = df.append({
          'vid_ID': vid_ID,
          'channel_title': channel_title,
          'title':title,
          'description': description,
          'view_count':view_count,
          'like_count': like_count,
          'comment_count':comment_count,
          'time_published':time_published
      }, ignore_index=True)

  return df

In [5]:
def getUploadResponses(num_pages):
  # get playlist ID
  url = 'https://www.googleapis.com/youtube/v3/channels?part=contentDetails&id='+CH_ID+'&key='+API_KEY
  res = requests.get(url).json()

  UploadID = res['items'][0]['contentDetails']['relatedPlaylists']['uploads']

  # API call
  pageToken = ''

  # initial call
  url = 'https://www.googleapis.com/youtube/v3/playlistItems?part=snippet%2CcontentDetails&maxResults=50&playlistId='+UploadID+'&key='+API_KEY+'&pageToken='+pageToken
  res = requests.get(url).json()
  time.sleep(1)

  # get next page token
  nextPageToken = res.get('nextPageToken')
  for x in range(num_pages - 1):
    # requests next page
    url = 'https://www.googleapis.com/youtube/v3/playlistItems?part=snippet%2CcontentDetails&maxResults=50&playlistId='+UploadID+'&key='+API_KEY+'&pageToken='+nextPageToken
    next_page = requests.get(url).json()
    time.sleep(1)

    # add items from both responses into one response
    res['items'] = res['items'] + next_page['items']

    # check if there is another page, if not break loop
    # else get nextPageToken
    if 'nextPageToken' not in next_page:
       break
    else:
        nextPageToken = next_page['nextPageToken']
  
  return res

In [6]:
# keys
API_KEY = ''

In [7]:
# dataframe to save parse video FOX NEWS info
df1 = pd.DataFrame(columns = ['vid_ID', 'channel_title', 'description', 'time_published', 'title', 'view_count', 'like_count', 'comment_count'])

In [8]:
# fetch responses for  CNN, build dataframe
CH_ID = 'UCupvZG-5ko_eiXAupbDfxWw'
CNNRes = getUploadResponses(20)
df1 = get_videos(CNNRes,df1)

In [13]:
# dataframe to save parse video CNN info
df2 = pd.DataFrame(columns = ['vid_ID', 'channel_title', 'description', 'time_published', 'title', 'view_count', 'like_count', 'comment_count'])

In [14]:
# fetch responses for Fox News, build dataframe
CH_ID = 'UCXIJgqnII2ZOINSWNOGFThA'
FxRes = getUploadResponses(20)
df2 = get_videos(FxRes,df2)

In [29]:
# concat dataframes
df = pd.concat([df1,df2], ignore_index=True)

## Clean Data

In [32]:
df

,vid_ID,channel_title,description,time_published,title,view_count,like_count,comment_count
0,enkiA-RmAK0,CNN,Members of the House Select Committee investig...,2021-12-17,Jan 6 investigators believe Nov. 4 text pushin...,73020,2007,1351
1,33X9-EEbJYU,CNN,Former police officer Kim Potter broke down on...,2021-12-17,Kim Potter breaks down on the stand during tes...,69997,676,2068
2,IeSo_o1O5J8,CNN,A House oversight committee report reveals tha...,2021-12-17,New damning report about Trump’s response to C...,350706,9515,6029
3,ESWScbGOB4I,CNN,Dr. Sanjay Gupta follows several families who ...,2021-12-17,‘Miracle’: See impact marijuana had on autisti...,76971,2962,1230
4,1aP7FE0ws0s,CNN,A judge in Delaware has found that Fox News' c...,2021-12-17,Dominion Voting wins key decision in lawsuit a...,285136,8202,3634
...,...,...,...,...,...,...,...,...
1995,D_dPUcQUlYU,Fox News,Nassau County district attorney-elect Anne Don...,2021-11-07,Republicans win big on Long Island,322232,13007,1706
1996,CEiX9LtkOnY,Fox News,Former Speaker of the House Newt Gingrich deta...,2021-11-07,Newt Gingrich explains why Democrats aren't th...,119722,4775,993
1997,yR1P-Dd-PCs,Fox News,‘The Big Saturday Show’ panelists try to make ...,2021-11-07,"The Travis Scott concert crowd surge tragedy, ...",322328,4281,3032
1998,Spp_psCj3l4,Fox News,'The Big Saturday Show' panelists weigh in on ...,2021-11-07,"'Big Saturday Show' praises Bill Maher, parent...",276497,9508,2382


In [31]:
# datatypes of columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   vid_ID          2000 non-null   object
 1   channel_title   2000 non-null   object
 2   description     2000 non-null   object
 3   time_published  2000 non-null   object
 4   title           2000 non-null   object
 5   view_count      2000 non-null   object
 6   like_count      2000 non-null   object
 7   comment_count   2000 non-null   object
dtypes: object(8)
memory usage: 125.1+ KB


**Cleaning by:**
1. Change time to correct data type
2. Make count columns into integers

In [33]:
# time_published to correct datetime format
df['time_published'] = pd.to_datetime(df['time_published'], format = '%Y/%m/%d')

In [36]:
df[['view_count', 'like_count', 'comment_count']] = df[['view_count', 'like_count', 'comment_count']].astype(float)

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   vid_ID          2000 non-null   object        
 1   channel_title   2000 non-null   object        
 2   description     2000 non-null   object        
 3   time_published  2000 non-null   datetime64[ns]
 4   title           2000 non-null   object        
 5   view_count      2000 non-null   float64       
 6   like_count      2000 non-null   float64       
 7   comment_count   2000 non-null   float64       
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 125.1+ KB


In [40]:
# Save csv to drive

drive.mount('/drive')
df.to_csv('/drive/My Drive/YoutubeAPI/YoutubeVids.csv')

Mounted at /drive
